## Basic Multi AI Agent Architecture

In [ ]:
import os
from typing import Annotated
from typing_extensions import TypedDict

from langchain_groq import  ChatGroq
from langchain.chat_models import init_chat_model
from langchain_core.tools import tool
from langchain_tavily import TavilySearch

from langgraph.graph import  StateGraph, START, END
from langgraph.graph.message import add_messages ## work add the msg appeneded to the state
from langgraph.prebuilt import ToolNode, tools_condition
from langgraph.types import Command, interrupt
from langgraph.checkpoint.memory import MemorySaver

from dotenv import load_dotenv
load_dotenv()

os.environ["LC_API_KEY"] = "true"

### state

In [ ]:


class AgentState(TypedDict):
    # Msg have type list defines how this state key should be updated
    next_agent : str # which agent need to go next
    # messages : Annotated[list, add_messages] ## appended to the list

# graph_builder =StateGraph(AgentState)

# graph_builder

In [6]:
websearch_tool =TavilySearch(max_results=2)


@tool
def write_summary(content:str)->str:
    """write summary from the given or provided content"""
    summary = f"summary of the finding {content[:500]}...."
    return summary

In [7]:
llm =ChatGroq(model="openai/gpt-oss-20b")
llm

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000002205E17A850>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000002205E17B250>, model_name='openai/gpt-oss-20b', model_kwargs={}, groq_api_key=SecretStr('**********'))

### Agents 

In [ ]:
def researcher_agent(state: AgentState):
    """Researcher agent that searches for information"""
    
    messages = state["messages"]
    
    # Add system message for context
    system_msg = SystemMessage(content="You are a research assistant. Use the search_web tool to find information about the user's request.")
    
    # Call LLM with tools
    researcher_llm = llm.bind_tools([search_web])
    response = researcher_llm.invoke([system_msg] + messages)
    
    # Return the response and route to writer
    return {
        "messages": [response],
        "next_agent": "writer"
    }

In [ ]:
def writer_agent(state: AgentState):
    """Writer agent that creates summaries"""
    
    messages = state["messages"]
    
    # Add system message
    system_msg = SystemMessage(content="You are a technical writer. Review the conversation and create a clear, concise summary of the findings.")
    
    # Simple completion without tools
    response = llm.invoke([system_msg] + messages)
    
    return {
        "messages": [response],
        "next_agent": "end"
    }